![Astrofisica Computacional](../logo.png)

# Example: Searching for Neighbor Points

Eduard Larrañaga (ealarranaga@unal.edu.co)

In [1]:
import numpy as np
#import sys

rng = np.random.default_rng(413)  # initialise our random number generator

We will create an array with the 3D-coordinates, randomly generated in the range \[0,1), of  5 points:

In [14]:
N = 5
dimension = 3
points = rng.random((N, dimension))
points

array([[0.31828742, 0.36930542, 0.54105854],
       [0.39276773, 0.62638356, 0.66135177],
       [0.58237962, 0.04456844, 0.97513086],
       [0.63687676, 0.48898044, 0.49416917],
       [0.66602975, 0.39266848, 0.51225112]])

In [5]:
points[1] # Coordinates of one of the points

array([0.59527896, 0.72462972, 0.93176844])

We want to find the nearest point for each one, assuming an Euclidean distance. We will use the function `numpy.linalg.norm` to calculate the distances.

Info: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.

The norm of the position of a single point is 

In [8]:
np.linalg.norm(points[1])

1.3219824872088564

The norm of all the points is given by

In [9]:
np.linalg.norm(points, axis=1)

array([1.27688138, 1.32198249, 0.67801184, 1.1384153 , 1.06183807,
       1.25817122])

Now we reshape the array of coordinates by introducing another dimension:

In [15]:
resh_points = points.reshape(N, 1, dimension)
resh_points

array([[[0.31828742, 0.36930542, 0.54105854]],

       [[0.39276773, 0.62638356, 0.66135177]],

       [[0.58237962, 0.04456844, 0.97513086]],

       [[0.63687676, 0.48898044, 0.49416917]],

       [[0.66602975, 0.39266848, 0.51225112]]])

This let us calculate the matrix of differences of coordinates

In [16]:
Delta_coordinates = resh_points - points
Delta_coordinates

array([[[ 0.        ,  0.        ,  0.        ],
        [-0.0744803 , -0.25707813, -0.12029323],
        [-0.2640922 ,  0.32473698, -0.43407232],
        [-0.31858934, -0.11967502,  0.04688937],
        [-0.34774233, -0.02336306,  0.02880742]],

       [[ 0.0744803 ,  0.25707813,  0.12029323],
        [ 0.        ,  0.        ,  0.        ],
        [-0.18961189,  0.58181512, -0.31377909],
        [-0.24410903,  0.13740311,  0.1671826 ],
        [-0.27326203,  0.23371507,  0.14910065]],

       [[ 0.2640922 , -0.32473698,  0.43407232],
        [ 0.18961189, -0.58181512,  0.31377909],
        [ 0.        ,  0.        ,  0.        ],
        [-0.05449714, -0.444412  ,  0.48096169],
        [-0.08365013, -0.34810004,  0.46287974]],

       [[ 0.31858934,  0.11967502, -0.04688937],
        [ 0.24410903, -0.13740311, -0.1671826 ],
        [ 0.05449714,  0.444412  , -0.48096169],
        [ 0.        ,  0.        ,  0.        ],
        [-0.02915299,  0.09631196, -0.01808195]],

       [[ 0.

In [46]:
# Note that the shapes change as
print(resh_points.shape, " - ", points.shape, " => ", Delta_coordinates.shape)

(4, 1, 3)  -  (4, 3)  =>  (4, 4, 3)


Now, we use the `numpy.linalg.norm()` function to calculate the matrix of distances:

In [17]:
distances = np.linalg.norm((points.reshape(N, 1, dimension) - points), axis=2)
distances

array([[0.        , 0.29343984, 0.60300711, 0.34354023, 0.34971478],
       [0.29343984, 0.        , 0.68769093, 0.32621903, 0.38926324],
       [0.60300711, 0.68769093, 0.        , 0.65711195, 0.58517402],
       [0.34354023, 0.32621903, 0.65711195, 0.        , 0.10223917],
       [0.34971478, 0.38926324, 0.58517402, 0.10223917, 0.        ]])

It is clear that the diagonal contains the information of the "self-distance". We correct these values in order to obtain the minimum values,

In [18]:
distances[np.diag_indices_from(distances)] = np.inf
distances

array([[       inf, 0.29343984, 0.60300711, 0.34354023, 0.34971478],
       [0.29343984,        inf, 0.68769093, 0.32621903, 0.38926324],
       [0.60300711, 0.68769093,        inf, 0.65711195, 0.58517402],
       [0.34354023, 0.32621903, 0.65711195,        inf, 0.10223917],
       [0.34971478, 0.38926324, 0.58517402, 0.10223917,        inf]])

Finally, we find the minimum values for each column, giving the nearest point list:

In [19]:
np.argmin(distances, axis=1)

array([1, 0, 4, 4, 3])

The above solution is summarised in the following three steps

In [20]:
distances = np.linalg.norm((points.reshape(N, 1, dimension) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
nearest_points = np.argmin(distances, axis=1)
nearest_points

array([1, 0, 4, 4, 3])

### Comparing to `scikit-learn`

Now we will use the built-in function `sklearn.neighbors.NearestNeighbors`.

In [21]:
from sklearn.neighbors import NearestNeighbors
_, indices = NearestNeighbors().fit(points).kneighbors(points, 2)
indices[:, 1]

array([1, 0, 4, 4, 3])

Lets measure the time to find the nearest points using both methods.

In [22]:
%timeit _, indices = NearestNeighbors().fit(points).kneighbors(points, 2)

81.7 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%%timeit
distances = np.linalg.norm((points.reshape(N, 1, dimension) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
np.argmin(distances, axis=1)

11.1 µs ± 69.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now we can increase the number of points...

In [24]:
N = 1000
dimension = 3
points = rng.random((N, dimension))
points

array([[0.57464125, 0.48227587, 0.42294799],
       [0.81566465, 0.28923267, 0.58365257],
       [0.28353119, 0.24542024, 0.70108304],
       ...,
       [0.50644148, 0.24502326, 0.04422695],
       [0.15914083, 0.66043912, 0.74649669],
       [0.11877764, 0.36874415, 0.94328799]])

In [26]:
%%timeit
distances = np.linalg.norm((points.reshape(N, 1, dimension) - points), axis=2)
distances[np.diag_indices_from(distances)] = np.inf
np.argmin(distances, axis=1)

21.1 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit _, indices = NearestNeighbors().fit(points).kneighbors(points, 2)

875 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
